In [16]:
import torch
import torch.nn as nn
import chess 
torch.cuda.empty_cache()

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
print(device)

cuda


In [18]:
# convolutional module 
class kcolbvonc(nn.Module):
    def __init__(self, ic, oc, ks, st, pd, active: bool = True, ip:bool = True):
        super(kcolbvonc, self).__init__()
        
        self.ip = ip
        self.tive = active
        self.kcolb = self.kcolbkcolb(ic, oc, ks, st, pd)
        
    def kcolbkcolb(self, ic, oc, ks, st, pd):
        
        if not self.tive:
            
            kcolbconv = nn.Sequential(
                nn.Conv2d(ic, oc, ks, st, pd), 
                nn.BatchNorm2d(oc), 
            )
        
        else:

            kcolbconv = nn.Sequential(
                nn.Conv2d(ic, oc, ks, st, pd), 
                nn.BatchNorm2d(oc), 
                nn.LeakyReLU(inplace = self.ip)
            )
            
        return kcolbconv
    
    def forward(self, idk) -> torch.Tensor:
        
        return self.kcolb(idk)

In [19]:
# residual block built using kcolbvonc 
class laud(nn.Module):
    def __init__(self, ic, oc, ks, st, pd, nnn: int, jump: bool = True):
        super(laud, self).__init__()
        
        self.ic = ic 
        # in channel 
        self.oc = oc
        # out channel 
        self.ks = ks 
        # kernel size 
        self.st = st
        # stride 
        self.pd = pd 
        # padding 
        self.j = jump 
        # enable skip connection or not 
        self.nnn = nnn 
        # number of layers consist in a residual block 
        self.nc: int = 0
        # operational channel between layers 
        
        self.reyal = self.laudreyal() 
        # internal layers construction 
        self.matcher = kcolbvonc(ic, oc, 1, 1, 0, active = False) 
        # channel matcher for the initial input 
        self.smatcher = nn.AdaptiveMaxPool2d((8, 8)) 
        # spatial dimension matcher for the output
        
    def forward(self, idk):
        # idk -> initial input 
        
        logitnotgood = idk
        # logitnotgood -> tensor for forward pass 
        
        # iterating to get the output without skip connection
        for layer in self.reyal:
            logitnotgood = layer(logitnotgood) 
        
        if self.j and (self.ic != self.oc):
            
            idk = self.matcher(idk)
            logitnotgood = self.smatcher(logitnotgood)
            # matching the input and output 
            
            logitgood = idk + logitnotgood 
            # skip connection 
                        
        elif self.j and (self.ic == self.oc):

            logitnotgood = self.smatcher(logitnotgood)
            
            logitgood = idk + logitnotgood
        
        else:
            
            logitgood = logitnotgood

        return logitgood

    def laudreyal(self):
            
        reyal = nn.ModuleList()
        self.nc = self.ic
        
        for nnnn in range(self.nnn):
            
            ncc = self.ic * (nnnn + 2)
            
            rayal = kcolbvonc(self.nc, ncc, self.ks, self.st, self.pd)

            reyal.append(rayal)
            self.nc = ncc


        rayalo = kcolbvonc(self.nc, self.oc, self.ks, self.st, self.pd)

        reyal.append(rayalo)
        
        return reyal

In [20]:
dummyo = torch.rand((2, 12, 8, 8), device = device)
dummyoo = torch.rand((2, 120, 8, 8), device = device) 
# test input 

In [21]:
# full model 
class cheslermod(nn.Module):
    def __init__(self): 
        super(cheslermod, self).__init__()
        
        self.alin = nn.Sequential(
            laud(ic = 120, oc = 256, ks = 2, st = 1, pd = 1, nnn = 2), 
            laud(ic = 256, oc = 128, ks = 2, st = 1, pd = 1, nnn = 2), 
        )
        
        self.stein = nn.Sequential(
            laud(ic = 12, oc = 128, ks = 2, st = 1, pd = 1, nnn = 2),
        )
        
        self.after = nn.Sequential(
            laud(ic = 128, oc = 32, ks = 2, st = 1, pd = 1, nnn = 2), 
            nn.Flatten(), 
            self.lfter(2048, 1024, True), 
            self.lfter(1024, 512, True), 
            self.lfter(512, 120, False)
        )
    
    def forward(self, oo, o):
        # merging board state and legal moves logits 
        
        i = self.alin(oo)
        ii = self.stein(o)
        
        z = i + ii 
        
        zz = self.after(z)
        
        return zz
    
    @staticmethod
    def lfter(ic, oc, nfl):
        #nfl -> not final layer 
        
        if nfl:
            
            l = nn.Sequential(
                nn.Linear(ic, oc), 
                nn.BatchNorm1d(oc), 
                nn.LeakyReLU(inplace = True)
            )        
        
        else:
        # final layer, consist of no batch norm and a different activation func 
        
            l = nn.Sequential(
                nn.Linear(ic, oc),
                nn.Softmax(dim = -1)
            )
            
        return l

In [22]:
cheslerdom = cheslermod().to(device)

In [23]:
a = cheslerdom(dummyoo, dummyo)
print(a.shape)
print(sum(a[-1]))
print(a[0])
print(torch.argmax(a[0])) 
# test 

torch.Size([2, 120])
tensor(1.0000, device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.0075, 0.0082, 0.0109, 0.0072, 0.0051, 0.0081, 0.0054, 0.0046, 0.0104,
        0.0065, 0.0086, 0.0071, 0.0069, 0.0065, 0.0173, 0.0087, 0.0057, 0.0106,
        0.0063, 0.0072, 0.0062, 0.0112, 0.0065, 0.0155, 0.0075, 0.0054, 0.0066,
        0.0147, 0.0096, 0.0092, 0.0035, 0.0089, 0.0039, 0.0094, 0.0074, 0.0049,
        0.0104, 0.0057, 0.0106, 0.0072, 0.0042, 0.0093, 0.0093, 0.0069, 0.0083,
        0.0104, 0.0119, 0.0061, 0.0110, 0.0082, 0.0122, 0.0063, 0.0086, 0.0074,
        0.0080, 0.0066, 0.0085, 0.0065, 0.0085, 0.0057, 0.0075, 0.0081, 0.0058,
        0.0115, 0.0033, 0.0044, 0.0048, 0.0075, 0.0137, 0.0116, 0.0089, 0.0047,
        0.0073, 0.0059, 0.0085, 0.0079, 0.0093, 0.0107, 0.0052, 0.0096, 0.0063,
        0.0041, 0.0081, 0.0074, 0.0065, 0.0070, 0.0190, 0.0086, 0.0078, 0.0075,
        0.0076, 0.0094, 0.0090, 0.0061, 0.0078, 0.0118, 0.0142, 0.0107, 0.0082,
        0.0093, 0.0055, 0.0038, 0.0061, 0.0

In [24]:
def t(tensor):
    return torch.tensor(tensor, device = device, dtype = torch.float16) 

In [1]:
def LMs(board: chess.Board):

    legal_moves = list(board.legal_moves) 
    # fetching moves 
    legal_moves_san = [board.san(moves) for moves in legal_moves] 
    # listing moves in SAN 
    legal_moves_uci = [board.uci(moves) for moves in legal_moves] 
    # listing moves in UCI 

    return legal_moves_uci, legal_moves_san

NameError: name 'chess' is not defined

In [26]:
def ecode_board(board: chess.Board):

    board_enc = torch.zeros((12, 8, 8), device = device)
    # initialize the encoded board state, (12 -> 6 + 6 types of pieces (black and white), 8 -> vertical grids, 8 -> horizontal grids) 
    
    oh_label = {
        'P': 0,
        'R': 1,
        'N': 2,
        'B': 3,
        'Q': 4,
        'K': 5,
        'p': 6,
        'r': 7,
        'n': 8,
        'b': 9,
        'q': 10,
        'k': 11
    }
    # translating piece label to the correpond board state index 
    
    for square in chess.SQUARES: 
    
        piece = board.piece_at(square)
            
        if piece: 
        # checking if the square holds a piece 

            board_count = oh_label[str(piece)]
            # fetching for the board state idx 
            row_count, col_count = torch.tensor(divmod(square, 8), device = device) 
            # fetching for row & column idx 
            col_count = torch.abs(col_count - 7)
            
            
            board_enc[board_count, row_count, col_count] = 1
            # encode the correspond grid with 1 
            
    return board_enc

In [27]:
def ecode_moves(moves: list):

    state = torch.zeros((120, 8, 8), device = device) 
    # 120 -> max 120 moves 
    col_dict = {'a': 7, 'b': 6, 'c': 5, 'd': 4, 'e': 3, 'f': 2, 'g': 1, 'h': 0} 
    # indexing column 
    for ind, move in enumerate(moves[:120]):
        
        # fetching for coordinates 
        o_col = torch.tensor(col_dict[move[0]], device = device)
        # o_row = torch.abs(torch.tensor(int(move[1]) - 1, device = device) - 7)
        o_row = torch.tensor(int(move[1]) -1, device = device)
        e_col = torch.tensor(col_dict[move[2]], device = device)
        e_row = torch.tensor(int(move[3]) - 1, device = device)

        state[ind, o_row, o_col] = 1
        # encode the piece location with 1 
        
        if move[-1] == 'q': 
            # q -> promotion, target position = 3 if promotion 
            state[ind, e_row, e_col] = 3 
        else: 
            state[ind, e_row, e_col] = 2
            # standard target positions encoded with 2 
    return state

In [29]:
state = chess.Board() 
uci, san = LMs(state) 

print(ecode_moves(uci)[:5])

# raise SystemError 
# test 

tensor([[[0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [2., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 2., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 2., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
       

SystemError: 

In [ ]:
# penalty system 
def pan_sys(pred_ind, board_state: chess.Board, legal_uci, legal_san, v, captures):
    assert type(board_state) == chess.Board 

    if pred_ind >= len(legal_uci): 
        
        total_lm_penalty = torch.tensor(0, dtype = torch.float32, device = device)
        total_action_penalty = torch.tensor(0, dtype = torch.float32, device = device) 
        # legal move & choice penalty for choosing illegal moves 
        move = None 
        # no move returning 
        # v_val = None
    
    else: 
        # prioritizing capture if a legal move was chose 
        move = legal_uci[pred_ind]
        actionn = legal_san[pred_ind]
        action = actionn.replace('x', '').replace('+', '').replace('q', '').replace('#', '').replace('=', '') 
        # filter out useless symbols 
    
        file = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'] 
        # columns 
        s_piece = ['R', 'N', 'B'] 
        capture = 'x' 
        # capture notations 
    
        # fetching for the target grid 
        if 2 < len(action) < 6 and action[0] in file:
            action = f'p{action[1:3]}'
        if len(actionn) == 2:
            action = f'p{actionn}'
        elif len(actionn) >= 6:
            action = f'p{actionn[2:4]}'
        elif (capture in actionn) and (actionn[0] in s_piece) and (actionn[1] in file):
            action = actionn[:1] + actionn[2:]
        
        # indexing columns 
        col_dict = {'a': t(0), 'b': t(1), 'c': t(2), 'd': t(3), 'e': t(4), 'f': t(5), 'g': t(6), 'h': t(7)}
        
        # initial rewards for capture 
        penalty_ = {'p': t(2), 'n': t(4), 'b': t(4), 'r': t(7), 'q': t(10), 'k': t(float('inf'))}
    
        total_lm_penalty = torch.tensor(0.5, dtype = torch.float32, device = device)
        total_action_penalty = torch.tensor(0., dtype = torch.float32, device = device) 
        # initializing total penalties 
    
        if '#' in actionn:
            total_action_penalty = total_action_penalty + 3 
            # reward 3 points if checkmate 
    
        base_penalty = 0.5
        # setting the base penalty for calculation 
        
        if (capture not in actionn) and captures:
            total_action_penalty = total_action_penalty - 0 
            # calculating penalty if capturing is in the legal moves but no capture in the choice 
            # v_val = None
    
        elif  (capture not in actionn) and (not captures):
            total_action_penalty = torch.tensor(0, dtype = torch.float32, device = device) 
            # calculating penalty if no captures in the legal moves 
    
        else:
            try:
                # calculating the capture score for each type of piece 
                cap_loc = action[1:3]
                square = (torch.tensor(int(cap_loc[1]), device = device) - 1) * 8 + int(col_dict[cap_loc[0]])
                I_piece = action[0].lower() 
                # self 
                o_piece = str(board_state.piece_at(square.item())).lower() 
                # target 
                refI = penalty_[I_piece]
                refO = penalty_[o_piece]
                
                # prioritize capturing more valuable pieces 
                if refI < refO:
                    mul = (refO - refI) / 2
                    penalty_score = base_penalty * refO * mul
                elif refI > refO:
                    mul = (refI - refO)
                    penalty_score = base_penalty * (refO / mul)
                elif refI == refO:
                    penalty_score = base_penalty
                else:
                    penalty_score = 0
    
                total_action_penalty  = total_action_penalty + penalty_score
    
            except Exception as e:
                total_action_penalty = 0
    
            # v_val = bellman(v) 
    return total_action_penalty + total_lm_penalty #, move #, v_val 
    # total reward 

In [ ]:
def pg_loss(prob, r):
    if not prob == 0 or not prob.is_nan():
        loss = -torch.sum(torch.log(prob) * r)
    else:
        loss = None

    return loss

In [ ]:
from torch.distributions import Categorical

In [ ]:
# epsilon greedy choice, ex -> exploration 
def ep(prob_dist, true): 
    n = torch.rand(1) 
    ex = True
    if n >= 0.5: 
        move = torch.argmax(prob_dist, dim = -1) 
        ex = False
    else: 
        move = torch.randint(0, len(true), (1, prob_dist.size(0))).squeeze() 
        
    return move, ex 

In [ ]:
# update 
class renair: 
    def __init__(self, learning_rate): 
        
        self.dom = cheslermod().to(device) 
        # model 
        self.chesleropt = torch.optim.Adam(params=self.dom.parameters(), lr=learning_rate) 
        # optimizer 
        self.accurws = [] 
        # rewards after discount 
        self.accuprobs = None 
        # accumulate probabilities 
        self.gmrws = [] 
        # raw rewards 
        self.first = True 
        
    def update(self, stacked_moves, stacked_states, uci, san, caps, boards): 
        
        stigol = self.dom(stacked_moves, stacked_states) 
        # forward pass 
        # probs = Categorical(stigol) 
        inxs, exploration = ep(stigol, uci) 
        # getting index 
        zeroth = [0, 1, 2] 
        # batch idx 
        
        # collecting probabilities and rewards 
        if self.first: 
            self.accuprobs = stigol[zeroth, inxs] 
            self.first = False 
            
        elif not self.first: 
            self.accuprobs = torch.cat([self.accuprobs, stigol[zeroth, inxs]]) 
            
            
        for inx, board, cap in zip(inxs, boards, caps): 
            
            rw = pan_sys(inx, board, uci, san, 0, cap) 
            self.accurws.append(rw) 

        return inxs[-1] 
    
    def train(self, γ): 
        # backpropagate and update 
        for rw in self.accurws: 
            self.gmrws.append((rw * γ)) 
        # discount rewards 
        
        self.gmrws, self.accurws = t(self.gmrws), t(self.accurws) 
        self.accuprobs = torch.log(self.accuprobs)
        # log probs 
        
        loss = -(self.accuprobs * self.gmrws).mean() 
        # loss calculation 
        
        self.chesleropt.zero_grad() 
        loss.backward() 
        self.chesleropt.step() 
        
        self.accurws, self.gmrws = [self.accurws], [self.gmrws] 
        self.accurws.clear() 
        self.accuprobs = None 
        self.gmrws.clear() 
        self.first = True 
        # reinitialize params for the next iter 
        
        return loss.detach().item() 
        
renairt = renair(0.0001) 

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as mtpt

In [2]:
def train(epos):
    
    def stack(ten, dim = 0): 
        return torch.stack(ten, dim = dim) 
    # fast stacking 
        
    epos = range(epos) 
    losses = [] 
    c = 0
    low_loss = float('inf') 
    
    for epo in tqdm(epos): 
        
        if epo % 20 == 0 and epo != 0:
            mtpt.plot(losses)
            mtpt.title('Training Losses')
            mtpt.xlabel('Epochs')
            mtpt.ylabel('Loss')
            mtpt.grid(True)
            mtpt.show()
            
        envir = chess.Board() 
        # initialize the enviroment 
        init_legal_uci, init_legal_sann = LMs(envir) 
        # fetching for initial legal moves 
        init_move = True 
        
        total_moves = 0 
        
        acc_moves = [ecode_moves(init_legal_uci), ecode_moves(init_legal_uci)] 
        acc_states = [ecode_board(envir), ecode_board(envir)] 
        acc_boards = [envir, envir] 
        acc_caps = [False, False] 
        # batching inputs 
        
        while True: 
            
            mate = envir.is_checkmate() 
            draw = envir.is_stalemate() or envir.is_insufficient_material() 
            fin = mate or draw 
            # termination 
            
            if fin: 
                break

            legal_uci, legal_sann = LMs(envir) 

            captures = False 
            
            for legal_san in legal_sann:
                if 'x' in legal_san:
                    captures = True 
                    # checking if the move is a capture 
            
            state = ecode_board(envir) 
            moves = ecode_moves(legal_uci) 
            # getting encoded board state and legal moves 
            rand_move = legal_uci[torch.randint(0, len(legal_uci), (1, 1))] 
            move_rand = chess.Move.from_uci(rand_move) 
            # choosing a random move if model ouputs a illegal choice 
            
            if init_move: 
                acc_moves.append(moves) 
                acc_states.append(state) 
                acc_boards.append(envir)
                acc_caps.append(captures) 
                init_move = False 
                # mounting the first move to the batch 
            else: 
                acc_moves.pop(0), acc_states.pop(0), acc_boards.pop(0), acc_caps.pop(0) 
                acc_moves.append(moves) 
                acc_states.append(state) 
                acc_boards.append(envir) 
                acc_caps.append(captures) 
                # remove the earliest board state (2 moves behind) and append the last board state 

            cur_moves, cur_states, cur_boards, cur_cap = stack(acc_moves), stack(acc_states), acc_boards, acc_caps 
            # batching inputs 
            
            pred_move_inx = renairt.update(cur_moves, cur_states, legal_uci, legal_sann, cur_cap, cur_boards) 
            # getting the move index 
            
            # pushing a random move if the choice illegal, else push the chose move 
            if not pred_move_inx < len(legal_uci): 
                envir.push(move_rand) 
            else: 
                envir.push(chess.Move.from_uci(legal_uci[pred_move_inx])) 
        
        # 0.9 -> gamma rate, backpropagate 
        loss = renairt.train(0.9) 
        losses.append(loss) 
        
        # update weights and bias if loss improved 
        if loss < low_loss: 
            dom_state = renairt.dom().state_dict() 
            opt_state = renairt.chesleropt().state_dict() 
            
            torch.save(dom_state, '') 
            torch.save(opt_state, '') 
            low_loss = loss